In [ ]:
import os, sys, json, base64, datetime as dt
import pandas as pd
from typing import List, Dict, Tuple
from pathlib import Path
from dotenv import load_dotenv
from openai import OpenAI

In [ ]:
# 1) 환경 변수에서 API 키 읽기
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
def pair_images_from_dir(image_dir: str) -> List[Tuple[str, str]]:
    """
    • `image_dir` 안의 모든 .jpg/.jpeg/.png 파일을 이름순으로 정렬한 뒤,
      순서대로 2개씩 (앞면, 뒷면) 튜플 리스트로 반환한다.
    • 파일 개수가 홀수라면 마지막 하나는 무시한다.
    """
    p = Path(image_dir)
    imgs = sorted([str(f) for f in p.iterdir() if f.suffix.lower() in (".jpg", ".jpeg", ".png")])
    # 짝수 개수만 유지
    n = len(imgs) // 2 * 2
    # print(imgs) : 파일 경로를 str이 들어있는 리스트로 반환
    # -> ['img\\KakaoTalk_20250711_132102437.jpg', 'img\\KakaoTalk_20250711_132102437_01.jpg', 'img\\KakaoTalk_20250711_132102437_02.jpg', 'img\\KakaoTalk_20250711_132102437_03.jpg', 'img\\KakaoTalk_20250717_122102437.jpg', 'img\\KakaoTalk_20250717_122102437_01.jpg']

    paired_imgs = [(imgs[i], imgs[i + 1]) for i in range(0, n, 2)]
    # print(paired_imgs) : 파일 경로를 2개씩 짝지어서(튜플) 리스트로 반환
    # [('img\\KakaoTalk_20250711_132102437.jpg', 'img\\KakaoTalk_20250711_132102437_01.jpg'), ('img\\KakaoTalk_20250711_132102437_02.jpg', 'img\\KakaoTalk_20250711_132102437_03.jpg'), ('img\\KakaoTalk_20250717_122102437.jpg', 'img\\KakaoTalk_20250717_122102437_01.jpg')]
    return paired_imgs



In [ ]:
# 2) 이미지 → Base64 인코딩
def b64(path: str) -> str:
    with open(path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

In [ ]:
with open("img/KakaoTalk_20250711_132102437.jpg", "rb") as f:
    binary_data=f.read()
    binary_encoded_data=base64.b64encode(binary_data)
    binary_string_data=binary_encoded_data.decode("utf-8")
    
    print(binary_data[:100])
    print(binary_encoded_data[:100])
    print(binary_string_data[:100])

In [ ]:
# def call_openai_mydemo(front_img: str) -> Dict:

front_img="./img/KakaoTalk_20250711_132102437.jpg"
"""단일 영숮으 앞면만 추출 -> dict 반환"""
tools = [{
    "type": "function",
    "name": "parse_taxi_receipt",
    "description": "Extract key fields from Korean taxi receipts",
    "parameters": {
        "type": "object",
        "properties": {
            "date": {
                "type": "string",
                "description": "YYYY-MM-DD"
            },
            "applicant": {
                "type": "string",
                "description": "Hand-written name if present"
            },
            "purpose": {
                "type": "string",
                "description": "업무내용 필드, 예: '리사→집'"
            },
            "route": {
                "type": "string",
                "description": "출발지 - 도착지"
            },
            "fare": {
                "type": "integer",
                "description": "총 결제 금액 (원)"
            },
            "paid_at": {
                "type": "string",
                "description": "YYYY-MM-DD HH:MM"
            }
        },
        "required": ["date", "fare", "paid_at"],
        "additionalProperties": False
    }
}]
input=[
        {
            "role": "system",
            "content": "You are a helpful assistant that extracts structured data from Korean taxi receipts."
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "input_image",
                    "image_url": f"data:image/jpeg;base64,{b64(front_img)}"
                }
            ]
        },
    ]
response = client.responses.create(
    model="gpt-4o",
    input=
    tools=tools
)
print(response)


In [ ]:
if __name__ == "__main__":
    image_dir = "./img"
    for front, back in pair_images_from_dir(image_dir):
        print(front)    
        print(back)
        info = call_openai(front, back)
        print(info)

In [ ]:
if __name__ == "__main__":
    image_dir = "./img"
    for front, back in pair_images_from_dir(image_dir):
        print(front)    
        print(back)
        info = call_openai(front, back)
        print(info)

In [ ]:
if __name__ == "__main__":
    image_dir = "./img"
    for front, back in pair_images_from_dir(image_dir):
        print(front)    
        print(back)
        info = call_openai(front, back)
        print(info)

In [ ]:
if __name__ == "__main__":
    image_dir = "./img"
    for front, back in pair_images_from_dir(image_dir):
        print(front)    
        print(back)
        info = call_openai(front, back)
        print(info)

In [ ]:
if __name__ == "__main__":
    image_dir = "./img"
    for front, back in pair_images_from_dir(image_dir):
        print(front)    
        print(back)
        info = call_openai(front, back)
        print(info)

In [ ]:
if __name__ == "__main__":
    image_dir = "./img"
    for front, back in pair_images_from_dir(image_dir):
        print(front)    
        print(back)
        info = call_openai(front, back)
        print(info)

In [ ]:
if __name__ == "__main__":
    image_dir = "./img"
    for front, back in pair_images_from_dir(image_dir):
        print(front)    
        print(back)
        info = call_openai(front, back)
        print(info)

In [ ]:
if __name__ == "__main__":
    image_dir = "./img"
    for front, back in pair_images_from_dir(image_dir):
        print(front)    
        print(back)
        info = call_openai(front, back)
        print(info)